# Pandas와 PyMySQL 연동

In [ ]:
!pip install PyMySQL

In [ ]:
import pymysql
import pandas as pd

host_name = 'localhost' # 구글 컴퓨터라 내 컴퓨터의 localhost에 접속 불가 - 어디서나 접속할 수 있는 원격 DB가 필요하다
host_port = 3306
username = 'root'
password = '0000'
database_name = 'fisa'

db = pymysql.connect(
    host=host_name,     # MySQL Server Address
    port=host_port,          # MySQL Server Port
    user=username,      # MySQL username
    passwd=password,    # password for MySQL username
    db=database_name,   # Database name
    charset='utf8'
)


- 클라우드에 설치된 MySQL 8.0 SERVER


In [ ]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pymysql
import pandas as pd
import warnings  # 인터프리터가 내는 모든 경고메시지를 출력하지 않기
warnings.simplefilter(action='ignore') # Ignore warnings


host_name = ?
host_port = 3306
username = ?
password = ?
database_name = ?

conn = pymysql.connect(
    host=host_name,     # MySQL Server Address
    port=host_port,          # MySQL Server Port
    user=username,      # MySQL username
    passwd=password,    # password for MySQL username
    db=database_name,   # Database name
    charset='utf8mb4'
)

- pymysql의 연결 객체를 이용하여 pandas의 자료형인 데이터프레임 객체 생성하기

In [ ]:
import pandas as pd
df = pd.read_sql("select * from emp",conn) # read_csv 하면 csv 파일을 불러왔듯이 reqd_sql 함수
df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   empno     14 non-null     int64  
 1   ename     14 non-null     object 
 2   job       14 non-null     object 
 3   mgr       13 non-null     float64
 4   hiredate  14 non-null     object 
 5   sal       14 non-null     float64
 6   comm      4 non-null      float64
 7   deptno    14 non-null     int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 1.0+ KB


In [ ]:
pd.read_sql("show tables",conn)

,Tables_in_fisa
0,dept
1,emp
2,emp_new
3,salgrade
4,titanic_raw


- 접속정보는 중요하므로 별개 파일에 저장 (주로 json이나 yaml 사용)

In [ ]:
%%writefile db.yaml
HOST: ?
USER: ?
PASSWD: ?

Overwriting db.yaml


In [ ]:
import yaml
DB_INFO = "db.yaml"
with open(DB_INFO,"r") as f:
    db_info = yaml.load(f, Loader=yaml.Loader)

- DB 접속 정보

In [ ]:
db_info

NameError: name 'db_info' is not defined

In [ ]:
HOST = db_info["HOST"]
USER = db_info["USER"]
PASSWD = db_info["PASSWD"]
PORT = 3306

In [ ]:
import pymysql

conn = pymysql.connect(
    user = USER,
    passwd = PASSWD,
    host = HOST,
    port = PORT,
    db = 'fisa'
)
conn

In [ ]:
pd.read_sql_query("show tables",conn)


,Tables_in_fisa
0,dept
1,emp
2,emp_new
3,salgrade
4,titanic_raw


In [ ]:
df = pd.read_sql_query("select * from emp;",conn) # read_sql_table, read_sql_query
df

# df = pd.read_sql_table("emp",conn) # 향후 사용할 타 프레임워크와 호환되는 함수. pymysql에서는 작동하지 않음

In [ ]:
db.close() ### 이전 DB 커넥션을 끊어주세요

# DF를 데이터베이스 서버로 보내기

## SQLAlchemy
- python에서 사용하는 대표적인 ORM
- ORM(Object Relational Mapping) 이란?
    - 객체와 DB의 테이블이 매핑을 이루는 것을 말한다.
    - DB의 테이블 객체화 시켜서 데이터를 CRUD
    - SQL 을 직접 작성하지 않고 테이블을 조작할수 있다.
    - 사용하는 DBMS가 변경된다면 엔진만 바꿔주면 된다.
    - 쿼리 대신 메소드를 이용해서 CRUD 한다.

``` df.to_sql(name=테이블이름, con=engine, if_exists='append', index=False)```
-  Dataframe은 항상 index가 있기 때문에, 테이블 구조와 안맞을 수 있음, 그래서 index=False 로 작성
-  if_exists = 'fail' : 같은 이름의 Table이 존재할 경우 ValueError 가 남
   if_exists = 'replace': 같은 이름의 Table이 존재할 경우 기존 Table을 Drop하고 새로운 값을 Insert함
  if_exists = 'append': 같은 이름의 Table이 존재할 경우 기존 Table에 추가로 새로운 값을 Insert함
- 각 column name을 테이블의 컬럼명과 동일하게 하면 해당 컬럼에 데이터 입력
- empno 는 PRIMARY KEY로 AUTO_INCREMENT 옵션을 넣었으므로, 데이터 입력을 하지 않음

In [ ]:
!pip install sqlalchemy
!pip install pymysql

import pymysql
from sqlalchemy import create_engine  # InnoDB
import pandas as pd


## 방법은 거의
## 접속할DB종류+접속에사용할패키지명 + :// + userid:password @ 주소:포트번호/db?인코딩방법 의 형태를 띔
engine = create_engine(?)
df = pd.read_csv(?)
df.to_sql(name=?, con=engine, if_exists='append', index=False)

- DB 선택 없이 접속

In [ ]:
conn = pymysql.connect(
    user = USER,
    passwd = PASSWD,
    host = HOST,
    port = PORT,
    # autocommit = True # default : False # 오토커밋 기능 설정
)
conn

In [ ]:
conn.autocommit_mode

False

- DB 선택하기

In [ ]:
conn.select_db("fisa") # use DB명;

- 동적 SQL 사용

In [ ]:
# SQL문 실행 - %s 로 변하는 값이 들어갈 자리를 비워둠.


# 비워둔 자리 개수만큼 2번째 인자로 값을 튜플 안에 순서대로 전달


# 데이타 Fetch


- df를 csv 파일로 저장

- 사용후엔 꼭 connection을 닫아주세요!

In [ ]:
conn.close()